In [13]:
import pandas as pd
import os
import numpy as np
from Utils import Utils
from Sparse_Propensity_net import Sparse_Propensity_net

import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from Sparse_Propensity_net import Sparse_Propensity_net
from Utils import Utils

csv_path = "/Users/shantanughosh/Desktop/Shantanu_MS/Research/Mattia_Prosperi/Propensity_Dropout/Dataset/ihdp_sample.csv"

In [14]:
from dataloader import DataLoader
dL = DataLoader()
split_size = 0.8
np_covariates_X_train, np_covariates_X_test, np_covariates_Y_train, np_covariates_Y_test = dL.preprocess_data_from_csv(csv_path, split_size=0.8)
norm_X_train = np_covariates_X_train / np.linalg.norm(np_covariates_X_train)
ps_train_set = dL.convert_to_tensor(norm_X_train, np_covariates_Y_train)
device = Utils.get_device()
# data_loader = torch.utils.data.DataLoader(ps_train_set, shuffle=False, num_workers=4)
# for batch in data_loader:
#     covariates, treatment = batch
#     print(covariates.size())

.. Data Loading ..


In [ ]:
np_covariates_X_train

In [ ]:

norm_X_train

In [15]:

from Propensity_socre_network import Propensity_socre_network
from Sparse_Propensity_score import Sparse_Propensity_score

train_parameters_SAE = {
    "epochs": 100,
    "lr": 0.001,
    "batch_size": 32,
    "shuffle": True,
    "train_set": ps_train_set,
    "sparsity_probability": 0.5,
    "weight_decay": 0.0003,
    "BETA": 3,
    "model_save_path": "./Propensity_Model/SAE_PS_model_iter_id_" + str(1) + "_epoch_{0}_lr_{1}.pth"
}

ee= {}
ps_net_SAE = Sparse_Propensity_score()
print("############### Propensity Score SAE net Training ###############")
network = ps_net_SAE.train(train_parameters_SAE, device, phase="train")
network

############### Propensity Score SAE net Training ###############
.. Training started ..
Saved model path: ./Propensity_Model/SAE_PS_model_iter_id_1_epoch_100_lr_0.001.pth
----- Training SAE -----
Training mode: train
Epoch: 0, loss: 0.0197690259175081
Epoch: 1, loss: 0.005788279072332539
Epoch: 2, loss: 0.0013985584802484435
Epoch: 3, loss: 0.00018838017958363421
Epoch: 4, loss: 5.296804384239582e-05
Epoch: 5, loss: 3.161060139016052e-05
Epoch: 6, loss: 2.101035146264813e-05
Epoch: 7, loss: 1.5617353232013748e-05
Epoch: 8, loss: 1.2506361025244636e-05
Epoch: 9, loss: 1.082435957007539e-05
Epoch: 10, loss: 9.882592214700325e-06
Epoch: 11, loss: 9.430708288212037e-06
Epoch: 12, loss: 9.155635923793612e-06
Epoch: 13, loss: 9.097511888524567e-06
Epoch: 14, loss: 8.984028538275437e-06
Epoch: 15, loss: 8.906762990304024e-06
Epoch: 16, loss: 8.877330545051096e-06
Epoch: 17, loss: 8.860579902447122e-06
Epoch: 18, loss: 8.818506222795756e-06
Epoch: 19, loss: 8.786440814315269e-06
Epoch: 20, lo

Sparse_Propensity_net(
  (encoder): Sequential(
    (0): Linear(in_features=25, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=25, bias=True)
    (3): Tanh()
  )
)

In [16]:
for batch in data_loader:
    n = network
    n.eval()
    covariates, treatment = batch
    print(covariates)
    covariates = covariates[:, :-2]
    _c1 = n(covariates)
    print(_c1)
    break

tensor([[-0.0062, -0.0051,  0.0057,  0.0061,  0.0003, -0.0027,  0.0051,  0.0000,
          0.0051,  0.0051,  0.0000,  0.0000,  0.0051,  0.0051,  0.0000,  0.0051,
          0.0000,  0.0051,  0.0051,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0157,  0.0289]])
tensor([[-3.4587e-03, -1.4572e-03,  3.2098e-03,  6.2878e-04, -1.1656e-03,
          9.1483e-04,  2.3980e-03, -3.5502e-06,  3.2823e-03,  1.9743e-03,
         -4.6209e-04,  1.5981e-03,  2.0354e-03,  6.9600e-03, -5.8692e-04,
          4.7818e-03,  3.2210e-03,  3.9235e-03,  1.5352e-03,  2.1204e-03,
          2.6815e-03, -2.4900e-05,  1.2981e-03, -2.8844e-04,  1.4103e-03]],
       grad_fn=<TanhBackward>)


In [9]:
classifier = nn.Sequential(*list(network.children())[:-1])
classifier

Sequential(
  (0): Sequential(
    (0): Linear(in_features=25, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): Tanh()
  )
)

In [10]:

def _eval(eval_parameters, device, phase):
    print(".. Propensity score evaluation started using Sparse AE BAL ..")
    eval_set = eval_parameters["eval_set"]
    model_path = eval_parameters["model_path"]
    
#     network = Sparse_Propensity_net(training_mode=phase, device=device).to(device)
#     network.load_state_dict(torch.load(model_path, map_location=device))
    classifier.eval()
    print("chutiya")
    data_loader = torch.utils.data.DataLoader(eval_set, shuffle=False, num_workers=4)
    total_correct = 0
    eval_set_size = 0
    prop_score_list = []
    
    low_dim_X = []
    low_dim_treatment_Y = []
    for batch in data_loader:
#         print(batch)
#         print(type(batch))
        covariates, treatment = batch
        #print(covariates.size())
        covariates = covariates.to(device)
        covariates = covariates[:, :-2]
    
        eval_set_size += covariates.size(0)
        lower_dimensional_representation = classifier(covariates)
#         print(covariates.size())
        s = [element.item() for element in lower_dimensional_representation.flatten()]
        
        low_dim_X.append(s)
        low_dim_treatment_Y.append(treatment.item())
    
    # print("Accuracy: {0}".format(total_correct))
    print(".. Propensity score evaluation completed Sparse AE ..")

    print(eval_set_size)
    return low_dim_X, low_dim_treatment_Y

In [9]:
import numpy as np
eval_parameters_SAE = {
        "eval_set": ps_train_set,
        "model_path": "./Propensity_Model/SAE_PS_model_iter_id_{0}_epoch_200_lr_0.001.pth".format(1)
}
print(device)
low_dim_X, low_dim_treatment_Y = _eval(eval_parameters_SAE, device, phase="eval")
# print(len(low_dim_X))
# print(low_dim_treatment_Y)
np_X_treatment = np.array(low_dim_X)
np_Y_treatment = np.array(low_dim_treatment_Y)
low_dim_X

cpu
.. Propensity score evaluation started using Sparse AE BAL ..
chutiya
.. Propensity score evaluation completed Sparse AE ..
597


[[-0.9995726943016052,
  -0.9996437430381775,
  -0.9995710849761963,
  -0.9996670484542847,
  -0.999533474445343,
  -0.9996771216392517,
  -0.9992814660072327,
  -0.9994992613792419,
  -0.9996739029884338,
  -0.9993892908096313],
 [-0.9995486736297607,
  -0.9996244311332703,
  -0.9995476007461548,
  -0.9996492862701416,
  -0.9995073676109314,
  -0.9996590614318848,
  -0.9992420673370361,
  -0.9994717836380005,
  -0.9996559023857117,
  -0.9993583559989929],
 [-0.9995545148849487,
  -0.9996290802955627,
  -0.9995530843734741,
  -0.9996535778045654,
  -0.9995137453079224,
  -0.9996628165245056,
  -0.9992539286613464,
  -0.9994786977767944,
  -0.9996604919433594,
  -0.9993646740913391],
 [-0.9995496273040771,
  -0.9996264576911926,
  -0.9995494484901428,
  -0.9996499419212341,
  -0.9995094537734985,
  -0.9996598958969116,
  -0.9992457628250122,
  -0.9994747638702393,
  -0.999657392501831,
  -0.9993619918823242],
 [-0.9995797276496887,
  -0.999649703502655,
  -0.9995776414871216,
  -0.99967

In [10]:
print(np_X_treatment.shape)
print(np_Y_treatment.shape)

(597, 10)
(597,)


In [11]:
dL = DataLoader()
ps_low = dL.convert_to_tensor(np_X_treatment, np_Y_treatment)
ps_low

In [23]:

classifier.add_module('classifier', nn.Sequential(nn.Linear(in_features=10, out_features=2), nn.LogSoftmax(dim=1)))
classifier

Sequential(
  (0): Sequential(
    (0): Linear(in_features=25, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=10, out_features=2, bias=True)
    (1): LogSoftmax()
  )
)

In [11]:
def train(eval_parameters, device, phase):
    print(".. Propensity score evaluation started using Sparse AE..")
    train_set = eval_parameters["eval_set"]
    for n in classifier.parameters():
        print(n)
    # classifier.train()
    # classifier[0][0].weight.requires_grad = False
    # classifier[0][0].bias.requires_grad = False
    # classifier[0][2].weight.requires_grad = False
    # classifier[0][2].bias.requires_grad = False

    print("chutiya")
    total_correct = 0
    eval_set_size = 0
    prop_score_list = []

    low_dim_list = []
    data_loader = torch.utils.data.DataLoader(train_set, batch_size=32,
                                              shuffle=True, num_workers=4)
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=0.01)
    for epoch in range(100):
        classifier.train()
        total_loss = 0
        total_correct = 0
        train_set_size = 0

        for batch in data_loader:
            covariates, treatment = batch
            covariates = covariates.to(device)
            train_set_size += covariates.size(0)

            #             print(covariates.size())
            #             print(treatment.size())

            treatment = treatment.squeeze().to(device)
            covariates = covariates[:, :-2]

            treatment_pred = classifier(covariates)
            # print(treatment_pred)
            # print(F.softmax(treatment_pred))

            loss = criterion(treatment_pred, treatment).to(device)

            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(treatment_pred.data, 1)
            total_correct += torch.sum(preds == treatment.data)

        pred_accuracy = total_correct.item() / train_set_size
        print("Epoch: {0}, loss: {1}, correct: {2}/{3}, accuracy: {4}".
              format(epoch, total_loss, total_correct, train_set_size, pred_accuracy))
    #     for batch in eval_set:
    #         covariates, treatment = batch
    #         covariates = covariates.to(device)
    #         eval_set_size += covariates.size(0)
    #         treatment_pred = network(covariates)
    #         total_correct += Utils.get_num_correct(treatment_pred, treatment)

    #         treatment_pred_prob = F.softmax(treatment_pred, dim=1)
    #         treatment_pred_prob = treatment_pred_prob.squeeze()
    #         prop_score_list.append(treatment_pred_prob[1].item())

    return classifier

In [12]:
# print(type(ps_score_list_SAE))
# print(ps_score_list_SAE)
eval_parameters_SAE_1 = {
        # "eval_set": ps_low
        "eval_set": ps_train_set
    
        
}
print(device)
classifier = train(eval_parameters_SAE_1, device, phase="eval")

cpu
.. Propensity score evaluation started using Sparse AE..
Parameter containing:
tensor([[ 2.3404e-01,  1.1998e-01, -3.9976e-02, -3.4178e-02,  6.2337e-01,
         -8.1339e-02,  1.8522e-01,  1.0063e-01, -8.6488e-03,  2.5544e-02,
          1.0642e-01, -2.1345e-02, -1.5352e-01,  1.1219e-02,  5.5300e-02,
          1.7288e-01,  8.9442e-03, -1.2093e-01,  2.5051e-01, -1.7379e-01,
          4.7111e-02, -1.9492e-01,  3.4015e-01, -4.3636e-02,  6.3466e-02],
        [ 3.3928e-01,  5.2420e-01, -4.6211e-01, -9.6171e-01,  2.9718e-01,
         -9.8714e-01,  1.5481e-01, -9.1539e-03, -7.7178e-03, -1.4553e-01,
          1.7216e-01,  1.9995e-01, -3.5846e-03, -1.4927e-01, -6.5919e-02,
         -8.4236e-02, -1.3061e-01, -1.6517e-01, -9.1162e-03, -1.0430e-01,
          6.7379e-02,  2.5677e-01, -1.5798e-01,  6.5582e-02, -1.4660e-01],
        [ 5.2453e-02, -1.1587e-01,  4.2172e-02,  3.3135e-01, -1.0463e-01,
          1.5739e-01, -6.3417e-02,  1.6346e-02, -1.1379e-01,  4.3921e-02,
         -1.5626e-01,  1.34

Epoch: 0, loss: -10.900599971413612, correct: 401/597, accuracy: 0.6716917922948074
Epoch: 1, loss: -18.270769238471985, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 2, loss: -18.895485043525696, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 3, loss: -18.946717083454132, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 4, loss: -18.961297929286957, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 5, loss: -18.96908289194107, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 6, loss: -18.974529504776, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 7, loss: -18.97870934009552, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 8, loss: -18.98196417093277, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 9, loss: -18.98451030254364, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 10, loss: -18.986503839492798, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 11, loss: -18.98816066980362, correct: 485/597, accuracy: 0.812395309882747

Epoch: 97, loss: -18.999698102474213, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 98, loss: -18.999704837799072, correct: 485/597, accuracy: 0.8123953098827471
Epoch: 99, loss: -18.99971103668213, correct: 485/597, accuracy: 0.8123953098827471
